(Optional) This notebook was used to generate the test scenario. Feel free to modify and run this notebook if you would like to generate your own similar test scenario.


In [1]:
from utils import sedaroLogin
import numpy as np
import pandas

In [2]:
sedaro = sedaroLogin()

In [4]:
# Set to the id of your scenario branch
SCENARIO_BRANCH_ID = ''

scenario = sedaro.scenario(SCENARIO_BRANCH_ID)

Create ground agents and an agent group for the train stations in reference_data/stations.csv

In [8]:
stations = pandas.read_csv('reference_data/stations.csv')
kinematics = scenario.EcefStationaryKinematics.create() # This will make the ground agent fixed in ECEF
# Helper fn
def get_lat(lat_str:str) -> float:
    s = lat_str.split('°')
    if s[1] == ' N':
        return float(s[0])
    else:
        return -float(s[0])
def get_lon(lon_str:str) -> float:
    s = lon_str.split('°')
    if s[1] == ' E':
        return float(s[0])
    else:
        return -float(s[0])

# Add stations as agent and as tg
crud = []
names = []
for _, row in stations.iterrows():
    name = row['Train Station']
    if name in names:
        name = row['City'] + ' ' + name
    names.append(name)
    crud.append({
        'name': name[:32],
        'type': 'PeripheralGroundPoint',
        'lat': {'deg': get_lat(row['Latitude'])},
        'lon': {'deg': get_lon(row['Longitude'])},
        'alt': {'km': 0},
        'id': f'${name}',
        'kinematics': kinematics.id,
    })
crud.append({
    'name': 'Some Train Stations',
    'type': 'AgentGroup',
    'agentType': 'GroundTarget',
    'agentAssociations': {f'${name}': {'priority': i} for i, name in enumerate(names)},
})
_ = scenario.crud(blocks=crud)

Create copies of an existing agent with differing orbital inclinations. I tinkered with RAAN as well so the spacecraft would start in the Eastern hemisphere.

In [ ]:
base_agent = scenario.TemplatedAgent.get_first()
base_orbit = base_agent.kinematics
orbit_params = base_orbit.initialStateDefParams

for i, inc in enumerate(np.linspace(0, 170, 10)):
    new_orbit = base_orbit.clone()
    new_orbit = new_orbit.update(initialStateDefParams={
        **orbit_params,
        'inc': inc,
        'raan': 90,
    })
    new_agent = base_agent.clone().update(kinematics=new_orbit.id, name=f'Agent {i}')